

<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/ISOTIPO-Color.jpg" title="Title text" width="20%" />

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-479 Reconocimiento de patrones en minería de datos </h1>

<H2 align='center'> Tarea 3 - Sistemas Recomendadores </H2>
<H3 align='center'> Lucio Fondón Rebolledo - 201773610-0</H3>
<H3 align='center'> Francisco Reyes Jainaga - 201773529-5</H3>
<hr style="height:2px;border:none"/>

# Instrucciones

1. La tarea debe ser desarrollada en parejas.  
2. Los medios de consultas serán Aula y el canal de Discord oficial del curso. 
3. La tarea debe ser realizada en Jupyter Notebook (Python3) utilizando este archivo como base. 
4. Se evaluará la correcta implementación de los algoritmos de filtrado colaborativo aplicados a los datasets escogido por cada grupo, como también la comprensión de los fundamentos teóricos, ventajas y desventajas de cada técnica. 
5. El archivo de entrega debe denominarse T3_Apellido1_Apellido2.ipynb. De no respetarse este formato existirá un descuento de 30 puntos. 
6. La fecha de entrega es el lunes 2 de agosto a las 23:55 hrs, posterior a esto se descontará 1 punto por cada minuto de atraso.  

In [7]:
#Librerías 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


## Datasets Tarea

Para la tarea tendrán a su disposición los siguientes 3 datasets, <b> de las cuales tendrán que escoger 2 para realizar la implementación de su tarea. </b>

   ### 1. MovieLens: 
   Es uno de los datasets más populares de recomendación y recopila las preferencias de usuarios de internet con respecto a películas que son evaluadas de 0 a 5 estrellas. Este dataset ha sido utilizado en diversos estudios de investigación en áreas como la recomendación personalizada y la psicología social.  
* <b>Archivos: </b>
    - ML_ratings.csv: contiene los ratings dados por usuarios a películas. Se compone por las columnas user_id, movie_id y rating. 
    - movies.csv: contiene la metadata sobre las películas. Incluye las columnas movie_id, title y genres (donde se encuentran los géneros de la película separados por "|"). 
 
* <b>Cantidad de ratings</b>: 100836
* <b>Cantidad de usuarios</b>: 610
* <b>Cantidad de películas</b>: 9724
* <b>Calificación</b> 0 a 5 (valores enteros) 

Referencias: [Movielens Dataset](https://grouplens.org/datasets/movielens/) 
   

In [8]:
df_ML_ratings = pd.read_csv("ML_ratings.csv")
df_movies = pd.read_csv("movies.csv")

display(df_ML_ratings)
display(df_movies)

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### 2. Book-Crossing: 
Es un datasets de ratings de libros recolectado por Cai-Nicolas Ziegler desde la <i>Book-Crossing community</i>. El dataset original contiene 1,149,780 ratings generados por 278,858 usuarios a 271,379 libros. 

* <b>Archivos</b>: 
    - BX_ratings.csv: contiene los ratings dados por usuarios a diferentes libros. Se compone de las columnas user_id, ISBN (identificador para los libros) y rating.
    - books.csv: contiene los títulos de libros asociados a los códigos ISBN (utilizados como item_id). Se compone de las columnas ISBN y title. 

* <b>Cantidad de ratings</b>: 359263
* <b>Cantidad de usuarios</b>: 10775
* <b>Cantidad de libros</b>: 10773
* <b>Calificación</b>: 0 a 10 (valores enteros)

Referencias: [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

In [9]:
df_BX_ratings = pd.read_csv("BX_ratings.csv")
df_books = pd.read_csv("books.csv")

display(df_BX_ratings)
display(df_books)

,user_id,ISBN,rating
0,276762,034544003X,0
1,276762,0380711524,5
2,276762,0451167317,0
3,276762,3404611306,0
4,276798,3442437407,0
...,...,...,...
359258,276704,0441007813,0
359259,276704,0446353957,0
359260,276704,0446605409,0
359261,276704,0743211383,7


,ISBN,title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi
...,...,...
271355,0440400988,There's a Bat in Bunk Five
271356,0525447644,From One to One Hundred
271357,006008667X,Lily Dale : The True Story of the Town that Ta...
271358,0192126040,Republic (World's Classics)


### 3. Jester:     
Es un dataset desarrollado por Ken Goldberg y su equipo en la Universidad de Berkeley, el cual contiene alrededor de 6 millones de ratings con respecto a 150 chistes cortos. 
* <b>Archivos</b>:
    - JT_ratings.csv: contiene los ratings dados por usuarios a diferentes chistes cortos. Se compone por las columnas joke_id, user_id y rating. 
    - jokes.csv: contiene 

* <b>Cantidad de ratings</b>: 199900
* <b>Cantidad de usuarios</b>: 1999
* <b>Cantidad de chistes</b>: 100 
* <b>Calificación</b>: -10 a 10 (valores reales)
    
Referencias: [Jester Dataset](http://eigentaste.berkeley.edu/dataset/)

In [10]:
df_JT_ratings = pd.read_csv("JT_ratings.csv")
df_jokes = pd.read_csv("jokes.csv")

display(df_JT_ratings)
display(df_jokes)

,joke_id,user_id,rating
0,0,1,5.10
1,1,1,4.90
2,2,1,1.75
3,3,1,-4.17
4,4,1,5.15
...,...,...,...
199895,95,1999,-4.13
199896,96,1999,-1.21
199897,97,1999,5.29
199898,98,1999,-9.13


,joke_id,joke_text
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...
...,...,...
95,95,Two attorneys went into a diner and ordered tw...
96,96,A teacher is explaining to her class how diffe...
97,97,Age and Womanhood\n\n1. Between the ages of 13...
98,98,A bus station is where a bus stops.\nA train s...


## Instrucciones

<img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11227-020-03266-2/MediaObjects/11227_2020_3266_Fig1_HTML.png" title="Title text" width="60%" />
<center> <i> Figura 1. Collaborative Filtering. </i> </center>

## 1. User based Collaborative Filtering (40 puntos) 

La primera parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en usuarios utilizando los k vecinos más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre el usuario objetivo y el resto de usuarios. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Es importante que en su procedimiento se muestren las ids de los k vecinos más cercanos.  

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumenta este parámetro?
    
   >c. ¿Cuál era el porcentaje de <i>sparsity</i> de la matriz usuarios-items? ¿Cuáles son las desventajas de este enfoque? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>


## Desarrollo P1

Primero, definiremos funciones que nos servirán de ayuda para los dos datasets elegidos (__MovieLens__ y __Jester__). Definiremos una función que nos retorne los $k$ id's de los usuarios más similares respecto a un usuario objetivo y otra función que realizará la recomendación de las $n$ mejores películas para el usuario objetivo

In [11]:
np.random.seed(0) # Se setea una semilla para poder replicar resultados

# Creamos una función que nos retorne los usuarios más similares respecto a un usuario objetivo
# Se utilizará (como es pedido) la similaridad coseno como métrica

"""
input
user_id  : (int) id del usuario objetivo 
matrix   : (2D array) matriz de ratings entre los usuarios y los items con ratings
k        : (int) k vecinos más cercanos

output
users    : (1D array) arreglo con las id's de los k usuarios más cercanos al usuario objetivo
"""

def similar_users(user_id, matrix, k=3):
    user = matrix[matrix.index == user_id] # ratings usuario objetivo
    other_users = matrix[matrix.index != user_id] # ratings resto de usuarios
    
    # Calculamos la similaridad coseno entre el usuario objetivo y el resto de ellos
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    indices = other_users.index.tolist()
    
    # Diccionario donde la llave es el id de un usuario distinto al objetivo
    # el valor es la similaridad coseno del usuario con el usuario objetivo
    index_similarity = dict(zip(indices, similarities))
        
    # Ordenamos por similaridad de mayor a menor
    index_similarity_sorted = sorted(index_similarity.items(), key=lambda x:x[1], reverse=True)
    
    # Escogemos los k primeros vecinos más cercanos (son los con mayor similitud con el usuario objetivo)
    top_users_similarities = index_similarity_sorted[:k]
    
    # Obtenemos los id de los usuarios
    users = [u[0] for u in top_users_similarities]
    return users

# Creamos función que nos retorne los n items con mejor recomendación para un usuario objetivo

"""
input
user_id            : (int) id del usuario objetivo 
similar_user_ids   : (1D array) arreglo que contiene las id's de usuarios similares
matrix             : (2D array) matriz de ratings entre los usuarios y los items con ratings
dataset            : (string) string que corresponde al nombre del dataset que se utiliza
n                  : (int) n items a recomendar para el usuario objetivo user_id

output
items_prediction   : (array) arreglo con las mejores n predicciones para el usuario objetivo
"""

def recommend_item(user_id, similar_user_ids, matrix, dataset, n=5):
    # Obtenemos los vectores de los usuarios similares al usuario objetivo obtenidos con la función similar_users
    similar_users = matrix[matrix.index.isin(similar_user_ids)]

    # Calculamos el rating de los k usuarios más cercanos
    similar_users = similar_users.mean(axis=0)
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
        
    # Obtenemos el vector del usuario objetivo
    user_df = matrix[matrix.index == user_id]
    user_df_transposed = user_df.transpose()
    user_df_transposed.columns = ['rating']
    
    # Obtenemos los items que el usuario objetivo NO ha rateado
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    items_not_rated = user_df_transposed.index.tolist()
        
    # Filtramos el rating promedio de los usuarios de los items no rateados por el usuario objetivo
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(items_not_rated)]
   
    # Ordenamos el dataframe de mayor a menor rating y obtenemos los n primeros
    similar_users_df_ordered = similar_users_df.sort_values(by=['mean'], ascending=False)
    top_n_items = similar_users_df_ordered.head(n)
    top_n_items_indices = top_n_items.index.tolist()
    
    # Dependiendo del dataset usado, se utiliza el nombre correspondiente
    item_id = ''
    if dataset == 'movielens':
        item_id = 'movie_id'
        df_final = df_movies
    elif dataset == 'jester':
        item_id = 'joke_id'
        df_final = df_jokes
    
    # Se consulta al dataset con los top n items seleccionados
    items_prediction = df_final[df_final[item_id].isin(top_n_items_indices)]
    
    return items_prediction

## Desarrollo P2

Ahora, debemos ingresar un nuevo usuario al sistema y que califique 10 productos. Para esto, se realizará una función que realizará automáticamente esta calificación para los dos datasets escogidos. Éstos se escogerán de manera al azar, por lo que se seteará una semilla para poder replicar resultados y que la revisión sea más adecuada y fácil de realizar

In [12]:
"""
input
dataset : (string) string que corresponde al nombre del dataset que se utiliza

output
df      : (pandas dataframe) dataframe con las 10 calificaciones agregadas
"""

def rate_10_products(dataset):
    if dataset == 'movielens':
        df = df_ML_ratings.copy()
        
        # Sacamos el id nuevo que tendrá el usuario, se le sumará +1 al último que se encuentre en el df
        new_user_id = df.iloc[-1]['user_id'] + 1
        
        # Obtenemos todos los id de las películas
        movie_ids = df['movie_id'].unique()
        
        # Seleccionamos al azar 10 películas para calificarlas
        random_movies = np.random.choice(movie_ids, size=10, replace=False)
        
        # Generamos ratings de manera aleatoria en el rango de 0-5
        random_ratings = np.random.randint(0, 6, size=10)
        
        # Diccionario con todos los valores generados
        random_final = {
            'user_id' : [new_user_id] * 10,
            'movie_id' : random_movies,
            'rating' : random_ratings,
        }
        
        # Generamos un dataframe para los datos del usuario nuevo
        df_random_ratings = pd.DataFrame(random_final)
        
        # Agregamos los datos al df original
        df = df.append(df_random_ratings)
        
        # Retornamos el nuevo df con los datos del nuevo usuario
        return df
        
    elif dataset == 'jester':
        df = df_JT_ratings.copy()
        
        # Sacamos el id nuevo que tendrá el usuario, se le sumará +1 al último que se encuentre en el df
        new_user_id = df.iloc[-1]['user_id'] + 1
        
        # Obtenemos todos los id de las bromas
        joke_ids = df['joke_id'].unique()
        
        # Seleccionamos al azar 10 bromas para calificarlas
        random_jokes = np.random.choice(joke_ids, size=10, replace=False)
        
        # Generamos ratings de manera aleatoria en el rango de -10.0 a 10.0
        random_ratings = np.random.uniform(-10.0, 10.0, 10).round(decimals=2)
        
        # Diccionario con todos los valores generados
        random_final = {
            'joke_id' : random_jokes,
            'user_id' : [new_user_id] * 10,
            'rating' : random_ratings,
        }
        
        # Generamos un dataframe para los datos del usuario nuevo
        df_random_ratings = pd.DataFrame(random_final)
        
        # Agregamos los datos al df original
        df = df.append(df_random_ratings)
        
        # Retornamos el nuevo df con los datos del nuevo usuario
        return df
            

### MovieLens

Primero, utilizaremos la función $\texttt{rate_10_products}$ para crear un nuevo usuario al azar y que califique aleatoriamente 10 películas

In [13]:
movies_with_new_rating = rate_10_products('movielens')
movies_with_new_rating

,user_id,movie_id,rating
0,1.0,1,4.0
1,1.0,3,4.0
2,1.0,6,4.0
3,1.0,47,5.0
4,1.0,50,5.0
...,...,...,...
5,611.0,4827,0.0
6,611.0,148978,2.0
7,611.0,40148,0.0
8,611.0,5016,1.0


Luego, debemos generar la matriz de ratings entre los usuarios y las películas rateadas, de tal forma que quede una matriz cuadrada $n x n$, en donde $n$ serían la cantidad de usuarios en el dataset, y los valores de la matriz será el rating del usuario $i$ con la película $j$

In [14]:
rating_matrix_movies = movies_with_new_rating.pivot_table(index = 'user_id',columns = 'movie_id', values = 'rating').fillna(0)
rating_matrix_movies

movie_id,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608.0,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora, solo debemos utilizar las funciones realizadas anteriormente para poder obtener las 5 recomendaciones pedidas. Se probarán con distintos valores de $k$, con el fin de poder concluir respecto a la elección de $k$

### k = 3

In [38]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


[562.0, 202.0, 100.0]


,movie_id,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
945,1246,Dead Poets Society (1989),Drama
1005,1307,When Harry Met Sally... (1989),Comedy|Romance
1146,1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance
1445,1968,"Breakfast Club, The (1985)",Comedy|Drama


### k = 5

In [16]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=5)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


[562.0, 202.0, 100.0, 555.0, 268.0]


,movie_id,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
1005,1307,When Harry Met Sally... (1989),Comedy|Romance
1322,1784,As Good as It Gets (1997),Comedy|Drama|Romance
1445,1968,"Breakfast Club, The (1985)",Comedy|Drama


### k = 10


In [17]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=10)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')



[562.0, 202.0, 100.0, 555.0, 268.0, 597.0, 414.0, 368.0, 84.0, 66.0]


,movie_id,title,genres
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
1445,1968,"Breakfast Club, The (1985)",Comedy|Drama


### k = 30


In [28]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=30)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')



[562.0, 202.0, 100.0, 555.0, 268.0, 597.0, 414.0, 368.0, 84.0, 66.0, 603.0, 19.0, 51.0, 57.0, 469.0, 184.0, 369.0, 217.0, 50.0, 474.0, 221.0, 292.0, 305.0, 288.0, 534.0, 18.0, 448.0, 352.0, 249.0, 387.0]


,movie_id,title,genres
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2078,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery


### Jester

Análogo a lo anterior, se realizan los mismos pasos anteriores

In [27]:
jokes_with_new_rating = rate_10_products('jester')
jokes_with_new_rating

,joke_id,user_id,rating
0,0,1.0,5.10
1,1,1.0,4.90
2,2,1.0,1.75
3,3,1.0,-4.17
4,4,1.0,5.15
...,...,...,...
5,77,2000.0,9.89
6,66,2000.0,-0.20
7,33,2000.0,-8.88
8,10,2000.0,8.77


In [20]:
rating_matrix_jokes = jokes_with_new_rating.pivot_table(index = 'user_id',columns = 'joke_id', values = 'rating').fillna(0)
rating_matrix_jokes

joke_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
1.0,5.10,4.90,1.75,-4.17,5.15,1.75,4.76,3.30,-2.57,-1.41,...,5.34,-4.61,3.59,7.18,0.92,6.31,-4.95,-0.19,3.25,4.37
2.0,-8.79,-0.87,1.99,-4.61,5.39,-0.78,1.60,1.07,-8.69,-4.66,...,3.59,1.21,2.86,-0.05,-1.75,-1.02,-0.97,4.13,-1.84,2.96
3.0,-3.50,-2.91,-2.18,-0.10,7.52,1.26,-5.39,1.50,-8.40,4.37,...,1.84,-4.03,-1.41,1.65,-3.79,3.98,-6.46,-6.89,-2.33,-7.38
4.0,7.14,-3.88,-3.06,0.05,6.26,6.65,-7.52,7.28,-5.15,-7.14,...,-4.47,6.36,4.71,-5.19,6.26,3.93,-2.57,1.07,2.33,-0.34
5.0,-8.79,-0.58,-0.58,8.98,7.67,8.25,4.08,2.52,-9.66,2.48,...,-0.29,9.37,8.30,9.13,-3.45,9.13,9.17,9.17,9.08,8.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996.0,3.06,1.84,1.50,1.50,0.05,2.14,1.17,0.63,0.10,3.01,...,3.54,0.78,-0.78,-0.05,4.56,3.98,3.50,-5.00,3.30,3.98
1997.0,2.33,7.48,-1.89,6.60,1.80,-6.46,6.65,1.31,-9.42,1.75,...,0.00,2.91,7.82,-4.76,2.18,-1.60,-7.38,0.49,-2.09,5.53
1998.0,7.82,1.84,-1.70,-5.15,2.91,5.15,-0.34,-3.50,4.32,8.69,...,5.53,5.78,6.02,0.00,-1.26,8.11,-1.26,-1.26,0.15,1.70


### k = 3

In [25]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


[1861.0, 1398.0, 623.0]


,joke_id,joke_text
75,75,There once was a man and a woman that both go...
82,82,"What a woman says:\n\n""This place is a mess! ..."
86,86,"A man, recently completing a routine physical ..."
88,88,A radio conversation of a US naval \nship with...
91,91,Early one morning a mother went to her sleepin...


### k = 5

In [24]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=5)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


[1861.0, 1398.0, 623.0, 388.0, 695.0]


,joke_id,joke_text
75,75,There once was a man and a woman that both go...
80,80,An Asian man goes into a New York CityBank to ...
82,82,"What a woman says:\n\n""This place is a mess! ..."
88,88,A radio conversation of a US naval \nship with...
91,91,Early one morning a mother went to her sleepin...


### k = 10

In [23]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=10)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


[1861.0, 1398.0, 623.0, 388.0, 695.0, 611.0, 746.0, 1124.0, 1798.0, 1238.0]


,joke_id,joke_text
42,42,Arnold Swartzeneger and Sylvester Stallone are...
80,80,An Asian man goes into a New York CityBank to ...
82,82,"What a woman says:\n\n""This place is a mess! ..."
86,86,"A man, recently completing a routine physical ..."
91,91,Early one morning a mother went to her sleepin...


### k = 30


In [29]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=30)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')

[1861.0, 1398.0, 623.0, 388.0, 695.0, 611.0, 746.0, 1124.0, 1798.0, 1238.0, 1868.0, 526.0, 490.0, 35.0, 1158.0, 992.0, 1260.0, 1499.0, 415.0, 737.0, 189.0, 371.0, 449.0, 1090.0, 341.0, 835.0, 767.0, 395.0, 890.0, 1287.0]


,joke_id,joke_text
42,42,Arnold Swartzeneger and Sylvester Stallone are...
60,60,"During a recent publicity outing, Hillary snea..."
82,82,"What a woman says:\n\n""This place is a mess! ..."
91,91,Early one morning a mother went to her sleepin...
92,92,"Reaching the end of a job interview, the human..."


### k = 50


In [30]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=50)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')

[1861.0, 1398.0, 623.0, 388.0, 695.0, 611.0, 746.0, 1124.0, 1798.0, 1238.0, 1868.0, 526.0, 490.0, 35.0, 1158.0, 992.0, 1260.0, 1499.0, 415.0, 737.0, 189.0, 371.0, 449.0, 1090.0, 341.0, 835.0, 767.0, 395.0, 890.0, 1287.0, 943.0, 1283.0, 1464.0, 1478.0, 514.0, 1205.0, 696.0, 234.0, 1373.0, 1746.0, 398.0, 636.0, 1386.0, 1841.0, 1192.0, 1115.0, 963.0, 483.0, 1382.0, 465.0]


,joke_id,joke_text
42,42,Arnold Swartzeneger and Sylvester Stallone are...
60,60,"During a recent publicity outing, Hillary snea..."
82,82,"What a woman says:\n\n""This place is a mess! ..."
91,91,Early one morning a mother went to her sleepin...
92,92,"Reaching the end of a job interview, the human..."


## Desarrollo P3

    
   >c. ¿Cuál era el porcentaje de <i>sparsity</i> de la matriz usuarios-items? ¿Cuáles son las desventajas de este enfoque? 
   
   > __R:__ El <i>sparsity</i> de una matriz $A$ se define como:
   $$sparsity = 1 - \frac{cantidad\_no\_ceros(A)}{total\_elementos(A)}$$ 
   entonces, crearemos una función que nos ayudará a calcular lo anterior



In [31]:
"""
input
A        : (pandas datafrane) matriz de ratings en forma de dataframe

output
sparsity : (double) valor del sparsity de la matriz
"""
def sparsity(A):
    # Calculamos la cantidad de elementos no nulos de A
    cantidad_no_ceros = np.count_nonzero(A)
    
    # Calculamos el total de elementos de A
    total_elementos = A.size
    
    # Calculamos la sparsity
    sparsity = 1 - cantidad_no_ceros/total_elementos
    
    return sparsity

   
   > __Sparsity para MovieLens:__


In [32]:
sparsity(rating_matrix_movies)

0.9830267931740927

   > __Sparsity para Jester:__


In [33]:
sparsity(rating_matrix_jokes)

0.0036000000000000476

## 2. Item based Collaborative Filtering (40 puntos) 

La segunda parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en ítems utilizando los k ítems más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre los ítems. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Para poder llevar a cabo la recomendación recordar los siguientes pasos:
    >a. Se debe generar la matriz de similaridad entre productos basada en la similaridad coseno.
    
    >b. Para realizar la recomendación, se debe predecir el rating de todos los productos que el usuario nuevo del sistema no haya calificado aún. Para esto, se deben obtener los k ítems más cercanos al ítem a predecir y se debe predecir su rating en base a la <i> weighted sum </i> de los ratings de dichos k ítems. 
    
    >c. Una vez predecidos los ratings para todos los productos sin calificación, se deben recomendar al usuario los 5 productos con mayor rating predecido. 

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumento este parámetro?
    
   >c. ¿Cuáles son las ventajas de este enfoque respecto al anterior? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>

## 3. Conclusiones finales (20 puntos)

1. Analice los resultados obtenidos y contraste ambos enfoques (ubcf y ibcf). 
2. Analice ventajas y desventajas de ambos enfoques. 